#Preprocessing, Dictionary Building and Validation

In [0]:
import numpy as np
import re
import string
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# txt = open('/content/drive/My Drive/a1_d3.txt','r',encoding='UTF-8')

In [0]:
txt = open('/content/a1_d3.txt','r',encoding='UTF-8')

In [0]:
dataset = txt.readlines()

In [0]:
list_of_sentences = []
sentiments = []
accuracy = []
test_accuracy = []
precision = []
f_score = []
recall = []

In [0]:
for i in range(len(dataset)-1):
    temp=dataset[i].split('\t')
    list_of_sentences.append(temp[0])
    sentiments.append((int)(temp[1][0]))
sentences = list_of_sentences

Text Preprocessing

In [0]:
#Converting to lower case and removing numbers and to remove punctuations.
for i in range(len(sentences)):
  sentences[i] = sentences[i].lower()  # converting to lower case
  sentences[i] = re.sub(r'\d+', '', sentences[i])   # to remove digits
  sentences[i] = sentences[i].translate(str.maketrans('','', string.punctuation)) #remove punctuations or symbols

In [0]:
sentences, test_sentences, sentiments,test_sentiments = train_test_split(sentences, sentiments, test_size=0.2,random_state = 5, stratify = sentiments)

In [0]:
final_dict = {}
mx_accuracy = 0
for k in range(5):    #5 fold cross-validation
  validate_sentiment = sentiments[160*k:160*(k+1)]
  train_sentiment = sentiments[0:160*k]
  train_sentiment.extend(sentiments[160*(k+1):800])
  validate_sentence = sentences[160*k:160*(k+1)]
  train_sentence = sentences[0:160*k]
  train_sentence.extend(sentences[160*(k+1):800])
  dict_words = {}
  pos_cnt=0
  total_pos = 0
  total_neg = 0
  for x in range(len(train_sentence)):
    if train_sentiment[x]==1:
      pos_cnt+=1
  for i in range(len(train_sentence)):
    words = train_sentence[i].split(' ') #list of tokenized words
    stemmed_words = words

    unique_words = set(stemmed_words)    
    #So that a sentence contain only 1 unique word. (this is useful if a sentence contains more than 1 occurence of a word)
    for j in unique_words:
      if j not in dict_words.keys():
        dict_words.setdefault(j,[1,1])  # make a list of 2 elements using Laplace Smoothing. 
        total_pos+=1  # total_pos is the count of all word's positive sentiments that belong to the dictionary 
        total_neg+=1

      if train_sentiment[i] is 1:
        dict_words[j][0]+=1 
        total_pos+=1     
      elif train_sentiment[i] is 0:
        dict_words[j][1]+=1
        total_neg+=1
    
  # this is a dictionary of lists , in which 1st element of list represents count of positive sentiments
  # (sentences)that word belongs to and the 2nd element is the count of negative sentiments(sentences) that word belong to.

  ####################################VALIDATION DATA################################################
  pred_sentiments = []
  for i in range(len(validate_sentence)):
    words = validate_sentence[i].split(" ")
    stemmed_words = words
    unique_words = set(stemmed_words)
    pos_prob = pos_cnt/len(train_sentiment)  #class probability(prior)
    neg_prob = 1-pos_prob
    for j in unique_words:
      if j not in dict_words.keys():
        pos_prob*=(1/total_pos) 
        neg_prob*=(1/total_neg)
      else:  
        pos_prob*=((dict_words[j][0])/total_pos) 
        neg_prob*=((dict_words[j][1])/total_neg)

    if pos_prob>=neg_prob:
      pred_sentiments.append(1)
    else:
      pred_sentiments.append(0)   
  
  #############################Evaluation #####################  
  tp=0 #true positive
  tn=0 #true negative
  fp=0 #false positive
  fn=0 #false negative
  for x in range(len(pred_sentiments)):
    if pred_sentiments[x] == validate_sentiment[x] == 1:
      tp+=1
    if pred_sentiments[x] == validate_sentiment[x] == 0:
      tn+=1
    if pred_sentiments[x] == 1 and validate_sentiment[x] == 0:
      fp+=1
    if pred_sentiments[x] ==0 and validate_sentiment[x] == 1:
      fn+=1
  
  test_accuracy.append((tp+tn)/(tp+tn+fp+fn))
  precision.append(tp/(tp+fp))
  recall.append(tp/(tp+fn))
  f_score.append(2*precision[len(precision)-1]*recall[len(recall)-1]/(precision[len(precision)-1]+recall[len(recall)-1]))

  if test_accuracy[len(test_accuracy)-1]>mx_accuracy:
    mx_accuracy = test_accuracy[len(test_accuracy)-1]
    final_dict = dict_words

In [292]:
dict_words

{'': [32, 33],
 'first': [2, 11],
 'plugged': [1, 4],
 'razr': [2, 3],
 'grtting': [1, 2],
 'a': [59, 69],
 'deal': [3, 2],
 'was': [27, 29],
 'i': [85, 94],
 'until': [1, 2],
 'it': [69, 83],
 'thought': [2, 2],
 'into': [4, 6],
 'good': [43, 9],
 'at': [3, 10],
 'my': [33, 41],
 'phone': [47, 47],
 'vc': [1, 2],
 'and': [98, 74],
 'is': [73, 55],
 'from': [14, 11],
 'excellent': [22, 1],
 'display': [3, 1],
 'any': [10, 6],
 'as': [11, 13],
 'that': [17, 32],
 'year': [3, 4],
 'camera': [4, 4],
 'this': [66, 64],
 'great': [53, 3],
 'case': [14, 6],
 'wallet': [2, 1],
 'type': [2, 1],
 'the': [113, 128],
 'works': [31, 1],
 'battery': [12, 16],
 'quality': [19, 12],
 'poor': [1, 11],
 'sound': [14, 10],
 'headset': [18, 13],
 'dualpurpose': [2, 1],
 'mic': [1, 2],
 'doesnt': [3, 9],
 'work': [4, 18],
 'had': [13, 21],
 'bluetoooth': [1, 2],
 'some': [2, 5],
 'still': [4, 10],
 'comfortable': [8, 3],
 'way': [3, 6],
 'fits': [6, 2],
 'on': [22, 32],
 'not': [9, 60],
 'for': [40, 40],


In [293]:
  print("Accuracy on validation set ",test_accuracy)
  print("Precision ",precision)
  print("Recall ", recall)
  print("Fscore = ",f_score)

Accuracy on validation set  [0.81875, 0.8125, 0.84375, 0.74375, 0.7924528301886793]
Precision  [0.7857142857142857, 0.797752808988764, 0.8295454545454546, 0.651685393258427, 0.7857142857142857]
Recall  [0.8571428571428571, 0.8554216867469879, 0.8795180722891566, 0.8529411764705882, 0.8651685393258427]
Fscore =  [0.8198757763975155, 0.8255813953488371, 0.8538011695906433, 0.7388535031847134, 0.8235294117647058]


In [0]:
accuracy_mean = (np.array(test_accuracy)).mean()
accuracy_std = (np.array(test_accuracy)).std()
f_score_mean = (np.array(f_score)).mean()
f_score_std = (np.array(f_score)).std()

In [295]:
print("Final Accuracy Validation = ",accuracy_mean.round(3), '+/-',accuracy_std.round(3))
print("Final Validation F-Score = ",f_score_mean.round(3),'+/-',f_score_std.round(3))

Final Accuracy Validation =  0.802 +/- 0.034
Final Validation F-Score =  0.812 +/- 0.039


#Evaluation on Test Data

In [0]:
####################################TEST DATA################################################
pred_sentiments = []
for i in range(len(test_sentences)):
  words = test_sentences[i].split(" ")
  stemmed_words = words
  unique_words = set(stemmed_words)
  pos_prob = pos_cnt/len(train_sentiment)  #class probability(prior)
  neg_prob = 1-pos_prob
  for j in unique_words:
    if j not in final_dict.keys():
      pos_prob*=0.5   ## if word not present in dict, then assign a prob of 0.5 to both classes
      neg_prob*=0.5
    else:
      pos_prob*=((final_dict[j][0])/total_pos)
      neg_prob*=((final_dict[j][1])/total_neg)

  if pos_prob>=neg_prob:
    pred_sentiments.append(1)
  else:
    pred_sentiments.append(0)   

#############################Evaluation #####################  
tp=0 #true positive
tn=0 #true negative
fp=0 #false positive
fn=0 #false negative
for x in range(len(pred_sentiments)):
  if pred_sentiments[x] == test_sentiments[x] == 1:
    tp+=1
  if pred_sentiments[x] == test_sentiments[x] == 0:
    tn+=1
  if pred_sentiments[x] == 1 and test_sentiments[x] == 0:
    fp+=1
  if pred_sentiments[x] ==0 and test_sentiments[x] == 1:
    fn+=1

test_accuracy = ((tp+tn)/(tp+tn+fp+fn))
precision = (tp/(tp+fp))
recall = (tp/(tp+fn))
f_score = 2*precision*recall/(precision + recall)


In [297]:
  print("Accuracy on test set ",test_accuracy)
  print("Precision ",precision)
  print("Recall ", recall)
  print("Fscore = ",f_score)

Accuracy on test set  0.82
Precision  0.7857142857142857
Recall  0.88
Fscore =  0.830188679245283
